In [1]:
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
import ast
from collections import OrderedDict
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(18,6))
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

<Figure size 1296x432 with 0 Axes>

In [2]:
train = pd.read_csv('tmdb_5000_movies.csv', index_col=['id'])

In [3]:
train.sort_index()

,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,
5,4000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name"": ""Comedy""}]",NaN,"[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""name"": ""new year's eve""}, {""id"": 616, ""name"": ""witch""}, {""id"": 622, ""name"": ""bet""}, {""id"": 922, ""name"": ""hotel room""}, {""id"": 2700, ""name"": ""sperm""}, {""id"": 12670, ""name"": ""los angeles""}, {""id"": 160488, ""name"": ""hoodlum""}, {""id"": 187056, ""name"": ""woman director""}, {""id"": 198129, ""name"": ""episode film""}]",en,Four Rooms,It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.,22.876230,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"": ""A Band Apart"", ""id"": 59}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",1995-12-09,4300000,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Twelve outrageous guests. Four scandalous requests. And one lone bellhop, in his first day on the job, who's in for the wildest New year's Eve of his life.",Four Rooms,6.5,530
11,11000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""name"": ""Action""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.starwars.com/films/star-wars-episode-iv-a-new-hope,"[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ""name"": ""galaxy""}, {""id"": 4271, ""name"": ""hermit""}, {""id"": 10013, ""name"": ""death star""}, {""id"": 10016, ""name"": ""lightsaber""}, {""id"": 10527, ""name"": ""jedi""}, {""id"": 11107, ""name"": ""rescue mission""}, {""id"": 11195, ""name"": ""empire""}, {""id"": 11196, ""name"": ""rebellion""}, {""id"": 13194, ""name"": ""planet""}, {""id"": 155336, ""name"": ""smuggler""}, {""id"": 160134, ""name"": ""the force""}, {""id"": 161176, ""name"": ""space opera""}, {""id"": 163295, ""name"": ""galactic war""}, {""id"": 178710, ""name"": ""stormtrooper""}, {""id"": 178712, ""name"": ""totalitarianism""}]",en,Star Wars,Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.,126.393695,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",1977-05-25,775398007,121.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,6624
12,94000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751, ""name"": ""Family""}]",http://movies.disney.com/finding-nemo,"[{""id"": 494, ""name"": ""father son relationship""}, {""id"": 10026, ""name"": ""harbor""}, {""id"": 14785, ""name"": ""underwater""}, {""id"": 33759, ""name"": ""fish tank""}, {""id"": 33760, ""name"": ""great barrier reef""}, {""id"": 156948, ""name"": ""missing child""}, {""id"": 179430, ""name"": ""aftercreditsstinger""}, {""id"": 179431, ""name"": ""duringcreditsstinger""}, {""id"": 180557, ""name"": ""short term memory loss""}, {""id"": 180568, ""name"": ""clownfish""}, {""id"": 180574, ""name"": ""father son reunion""}, {""id"": 181068, ""name"": ""protective father""}]",en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexpectedly taken from his Great Barrier Reef home to a dentist's office aquarium. It's up to his worrisome father Marlin and a friendly but forgetful fish Dory to bring Nemo home -- meeting vegetarian sharks, surfer dude turtles, hypnotic jellyfish, hungry

In [4]:
# drop meaningless and duplicated columns
train.drop(columns=['homepage','original_title','overview','spoken_languages','status','tagline','title','keywords'],inplace=True)

In [5]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 19995 to 25975
Data columns (total 11 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
original_language       4803 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
vote_average            4803 non-null float64
vote_count              4803 non-null int64
dtypes: float64(3), int64(3), object(5)
memory usage: 450.3+ KB


,budget,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [6]:
# Remove instances when budget is less than 1000
train = train[train['budget'] >= 1000]

In [47]:
# Remove instances when revenue is less than 1000
train = train[train['revenue'] >= 1000]

In [49]:
#genres drop instances with missing values
train.dropna(axis=0, how='any',subset=['release_date'], inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3211 entries, 19995 to 9367
Data columns (total 11 columns):
budget                  3211 non-null int64
genres                  3211 non-null object
original_language       3211 non-null object
popularity              3211 non-null float64
production_companies    3211 non-null object
production_countries    3211 non-null object
release_date            3211 non-null object
revenue                 3211 non-null int64
runtime                 3211 non-null float64
vote_average            3211 non-null float64
vote_count              3211 non-null int64
dtypes: float64(3), int64(3), object(5)
memory usage: 301.0+ KB


In [50]:
#runtime replace missing value with average
train['runtime'].fillna(train['runtime'].mean(),inplace=True)

In [51]:
#normalization of popularity (TA said we may compare whether normalized popularity provides better accuracy or not)
train['popularity_norm'] = preprocessing.scale(train['popularity'])
train['popularity'].head()

id
19995     150.437577
285       139.082615
206647    107.376788
49026     112.312950
49529      43.926995
Name: popularity, dtype: float64

In [52]:
#convert date to year
train.release_date = pd.to_datetime(train.release_date)
train['year'] = pd.DatetimeIndex(train['release_date']).year

In [53]:
#convert date to month
train.release_date = pd.to_datetime(train.release_date)
train['month'] = pd.DatetimeIndex(train['release_date']).month

In [54]:
#convert date to date
train.release_date = pd.to_datetime(train.release_date)
train['date'] = pd.DatetimeIndex(train['release_date']).day

In [56]:
#convert date to quarter
train.release_date = pd.to_datetime(train.release_date)
train['quarter'] = pd.PeriodIndex(train.release_date, freq = 'Q')
train['quarter'] = train['quarter'].astype(str)
train['quarter'] = train['quarter'].str.slice(5, 6)
train['quarter'] = train['quarter'].astype(int)

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3211 entries, 19995 to 9367
Data columns (total 16 columns):
budget                  3211 non-null int64
genres                  3211 non-null object
original_language       3211 non-null object
popularity              3211 non-null float64
production_companies    3211 non-null object
production_countries    3211 non-null object
release_date            3211 non-null datetime64[ns]
revenue                 3211 non-null int64
runtime                 3211 non-null float64
vote_average            3211 non-null float64
vote_count              3211 non-null int64
popularity_norm         3211 non-null float64
year                    3211 non-null int64
month                   3211 non-null int64
date                    3211 non-null int64
quarter                 3211 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(7), object(4)
memory usage: 426.5+ KB


In [59]:
train.head()

,budget,genres,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,vote_average,vote_count,popularity_norm,year,month,date,quarter
id,,,,,,,,,,,,,,,,
19995,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",en,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id"": 574}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-12-10,2787965087,162.0,7.2,11800,3.348585,2009,12,10,4
285,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",en,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""name"": ""Second Mate Productions"", ""id"": 19936}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,6.9,4500,3.035064,2007,5,19,2
206647,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",en,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": 69434}]","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2015-10-26,880674609,148.0,6.3,4466,2.159637,2015,10,26,4
49026,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name"": ""Thriller""}]",en,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC Entertainment"", ""id"": 9993}, {""name"": ""Syncopy"", ""id"": 9996}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,7.6,9106,2.295929,2012,7,16,3
49529,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fiction""}]",en,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-03-07,284139100,132.0,6.1,2124,0.407730,2012,3,7,1


In [60]:
genres_counter_dict = {}

def genres_counter_lambda(genres):
    genres_list = list(ast.literal_eval(genres))
    for genres in genres_list:
        if genres['name'] in genres_counter_dict:
            genres_counter_dict[genres['name']] += 1
        else:
            genres_counter_dict.update({genres['name']: 1})

train['genres'].apply(genres_counter_lambda)
display(OrderedDict(sorted(genres_counter_dict.items(), key=lambda x: x[1], reverse=True)))
# plt.xticks(rotation=90)
# plt.bar(genres_counter_dict.keys(), genres_counter_dict.values())

OrderedDict([('Drama', 1430),
             ('Comedy', 1101),
             ('Thriller', 930),
             ('Action', 915),
             ('Adventure', 660),
             ('Romance', 569),
             ('Crime', 519),
             ('Science Fiction', 430),
             ('Family', 364),
             ('Fantasy', 341),
             ('Horror', 329),
             ('Mystery', 264),
             ('Animation', 186),
             ('History', 145),
             ('War', 119),
             ('Music', 111),
             ('Western', 57),
             ('Documentary', 37),
             ('Foreign', 4)])

In [65]:
# Rank of languages
train['original_language'].value_counts()

1     3086
0       31
2       25
4       15
10      13
6       13
5        7
3        6
9        5
7        5
8        5
Name: original_language, dtype: int64

In [64]:
# Change languages to int with Top 10 (1-10) and others (0)
x = {'en','fr','ru','es','hi','ja','it','ko','cn','zh'}
y = set(train['original_language'])
useless_lang_list = y - x

languages_dict = {'en':1,'fr':2,'ru':3,'es':4,'hi':5,'ja':6,'it':7,'ko':8,'cn':9,'zh':10}
train['original_language'].replace(to_replace = useless_lang_list, value = 0,inplace = True,regex=True)
train['original_language'].replace(languages_dict,inplace = True)

display(train['original_language'].value_counts())
train['original_language']

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [66]:
# One hot encoder
lang_dummy = pd.get_dummies(train['original_language'], columns =['en','fr','ru','es','hi','ja','it','ko','cn','zh'])
train = pd.concat([train,lang_dummy],axis=1,sort=True)

In [67]:
# Rename columns with abbreviations of language
train = train.rename(columns ={0:"others",1:"en",2:"fr",3:"ru",4:"es",5:"hi",6:"ja",7:"it",8:"ko",9:"cn",10:"zh"})
pd.set_option('display.max_columns', 500)
train

,budget,genres,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,vote_average,vote_count,popularity_norm,year,month,date,quarter,others,en,fr,ru,es,hi,ja,it,ko,cn,zh
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,4000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name"": ""Comedy""}]",1,22.876230,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"": ""A Band Apart"", ""id"": 59}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",1995-12-09,4300000,98.0,6.5,530,-0.173501,1995,12,9,4,0,1,0,0,0,0,0,0,0,0,0
11,11000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""name"": ""Action""}, {""id"": 878, ""name"": ""Science Fiction""}]",1,126.393695,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",1977-05-25,775398007,121.0,8.1,6624,2.684711,1977,5,25,2,0,1,0,0,0,0,0,0,0,0,0
12,94000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751, ""name"": ""Family""}]",1,85.688789,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2003-05-30,940335536,100.0,7.6,6122,1.560811,2003,5,30,2,0,1,0,0,0,0,0,0,0,0,0
13,55000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""name"": ""Romance""}]",1,138.133331,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",1994-07-06,677945399,142.0,8.2,7927,3.008854,1994,7,6,3,0,1,0,0,0,0,0,0,0,0,0
14,15000000,"[{""id"": 18, ""name"": ""Drama""}]",1,80.878605,"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name"": ""Jinks/Cohen Company"", ""id"": 2721}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",1999-09-15,356296601,122.0,7.9,3313,1.427998,1999,9,15,3,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365222,36000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name"": ""History""}]",9,19.167377,"[{""name"": ""Mandarin Films Distribution Co."", ""id"": 4054}]","[{""iso_3166_1"": ""HK"", ""name"": ""Hong Kong""}]",2015-12-19,156844753,105.0,6.5,379,-0.275906,2015,12,19,4,0,0,0,0,0,0,0,0,0,1,0
375290,4500000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 28, ""name"": ""Action""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name"": ""History""}]",5,2.551559,"[{""name"": ""T-Series"", ""id"": 3522}, {""name"": ""Hari Om Entertainment Co."", ""id"": 3942}, {""name"": ""Cape Of Good Films"", ""id"": 35063}]","[{""iso_3166_1"": ""IN"", ""name"": ""India""}]",2016-01-22,32000000,126.0,7.3,57,-0.734684,2016,1,22,1,0,0,0,0,0,1,0,0,0,0,0
376659,20000000,"[{""id"": 35, ""name"": ""Comedy""}]",1,42.512296,"[{""name"": ""STX Entertainment"", ""id"": 47729}, {""name"": ""Huayi Brothers Pictures"", ""id"": 78996}, {""name"": ""BillBlock Media"", ""id"": 90185}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2016-07-28,183936074,100.0,6.3,1252,0.368669,2016,7,28,3,0,1,0,0,0,0,0,0,0,0,0


In [0]:
# Remove empty string
train['production_countries'].replace(['[]'],np.NaN,inplace=True)
train.dropna(axis=0, how='any',subset=['production_countries'], inplace=True)
train['production_companies'].replace(['[]'],np.NaN,inplace=True)
train.dropna(axis=0, how='any',subset=['production_companies'], inplace=True)
train['genres'].replace(['[]'],np.NaN,inplace=True)
train.dropna(axis=0, how='any',subset=['genres'], inplace=True)
train.info()

In [0]:
#train['production_countries']  = train['production_countries'].str[1:-1]
#train['production_countries'].str.split('}', expand=True)
#train.production_countries.apply(pd.Series)

#train['production_countries'] = train['production_countries'].astype(str)
#train['production_countries'] = train['production_countries'].str.replace('{','[')
#train['production_countries'] = train['production_countries'].str.replace('}',']')
pd.DataFrame(train['production_companies'].values.tolist())

In [0]:
genres_counter_dict = {}

def genres_counter_lambda(genres):
    genres_list = list(ast.literal_eval(genres))
    for genres in genres_list:
        if genres['name'] in genres_counter_dict:
            genres_counter_dict[genres['name']] += 1
        else:
            genres_counter_dict.update({genres['name']: 1})

train['genres'].apply(genres_counter_lambda)
display(OrderedDict(sorted(genres_counter_dict.items(), key=lambda x: x[1], reverse=True)))
# plt.xticks(rotation=90)
# plt.bar(genres_counter_dict.keys(), genres_counter_dict.values())

In [0]:
train.to_csv('ver1.csv')

# Graphs and Charts

In [0]:
budget_boxplot = sns.boxplot(train['budget'])

In [0]:
popularity_boxplot = sns.boxplot(train['popularity'])
popularity_norm_boxplot = sns.boxplot(train['popularity_norm'])

In [0]:
runtime_boxplot = sns.boxplot(train['runtime'])

In [0]:
revenue_boxplot = sns.boxplot(train['revenue'])

In [0]:
train['budget'].hist()

In [0]:
train['original_language'].hist()

In [0]:
train['popularity'].hist()
train['popularity_norm'].hist()

In [0]:
train['runtime'].hist()

In [0]:
train['revenue'].hist()

In [0]:
train['quarter'].hist(grid=False)

In [0]:
plt.scatter(train['revenue'], train['budget'])

In [0]:
plt.scatter(train['revenue'], train['popularity'])